<a href="https://colab.research.google.com/github/mkbahk/DeepLearningwithKeras/blob/master/mnist_tpu_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import tensorflow as tf
from tensorflow import keras
import time
import os

In [57]:
print(tf.config.list_physical_devices("CPU"), "\n\n")
print(tf.config.list_physical_devices("GPU"), "\n\n")
print(tf.config.list_physical_devices("TPU"), "\n\n")

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')] 


[] 


[] 




In [58]:
start = time.time() # 시작 시간 저장

In [59]:
if tf.__version__[0] != '2':
    raise ImportError("TensorFlow 2 is required for this example")

In [60]:
# The input data and labels.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, x_test) = (x_train / 255.0, x_test / 255.0)

# Add a channels dimension.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [61]:
def create_train_dataset():
    print("==============================Processing Training DataSet==============================\n\n")
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(32, drop_remainder=True)
    train_ds = train_ds.map(lambda d, l: (tf.cast(d, tf.float32), tf.cast(l, tf.float32)))
    return train_ds.repeat()

def create_test_dataset():
    print("==============================Processing Test  DataSet==============================\n\n")
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(32, drop_remainder=True)
    test_ds = test_ds.map(lambda d, l: (tf.cast(d, tf.float32), tf.cast(l, tf.float32)))
    return test_ds.repeat()

In [62]:
# Create the model using the IPU-specific Sequential class instead of the
# standard tf.keras.Sequential class
def create_model():
    model = keras.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(10, activation='softmax')])
    return model

In [63]:
def main():
        # Create an instance of the model.
        print("==============================Building Model==============================\n\n")
        model = create_model()

        # Get the training dataset.
        print("==============================Getting Training DataSet==============================\n\n")
        ds1 = create_train_dataset()
        print("==============================Getting Test DataSet==============================\n\n")
        ds2 = create_test_dataset()

        # Train the model.
        print("==============================Start Model Training==============================\n\n")
        !echo $COLAB_TPU_ADDR
        # TPU gRPC 접근 URI
        TPU_PATH = f"grpc://{os.environ['COLAB_TPU_ADDR']}"

        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_PATH)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)

        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        with strategy.scope():
          model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = tf.keras.optimizers.SGD(), metrics=['accuracy'])
          model.fit(ds1, steps_per_epoch=2000, epochs=50)

          print("\n\n==============================Checking the result==============================\n\n")
          loss, accuracy = model.evaluate(ds2, steps=1000)
          print("Validation loss: {}".format(loss))
          print("Validation accuracy: {}%".format(100.0 * accuracy))
          print("\n\n==============================Job Done by CPU !!!==============================")

In [ ]:
if __name__ == '__main__':
    main()

==============================Building Model==============================


==============================Getting Training DataSet==============================


==============================Processing Training DataSet==============================


==============================Getting Test DataSet==============================


==============================Processing Test  DataSet==============================


==============================Start Model Training==============================


10.15.209.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.15.209.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.15.209.162:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/50
   1/2000 [..............................] - ETA: 11:41 - loss: 2.3603 - accuracy: 0.1562WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0017s vs `on_train_batch_end` time: 0.0114s). Check your callbacks.


2000/2000 [==============================] - 23s 12ms/step - loss: 0.6259 - accuracy: 0.8296
Epoch 2/50
2000/2000 [==============================] - 23s 12ms/step - loss: 0.2671 - accuracy: 0.9216
Epoch 3/50
2000/2000 [==============================] - 23s 12ms/step - loss: 0.2099 - accuracy: 0.9384
Epoch 4/50
2000/2000 [==============================] - 23s 11ms/step - loss: 0.1746 - accuracy: 0.9488
Epoch 5/50
2000/2000 [==============================] - 23s 11ms/step - loss: 0.1460 - accuracy: 0.9568
Epoch 6/50
2000/2000 [==============================] - 23s 12ms/step - loss: 0.1265 - accuracy: 0.9634
Epoch 7/50
 984/2000 [=============>................] - ETA: 11s - loss: 0.1178 - accuracy: 0.9663

In [ ]:
print("실행시간 :", time.time() - start,"(초)")  # 현재시각 - 시작시간 = 실행 시간